In [82]:
import math

from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger
from nltk.corpus import stopwords

filepath: str = "your filepath"

with open(filepath, "r") as file:
    text: str = file.read().replace("\n", " ")

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

stopwords = stopwords.words("russian")


def preprocess_text(paragraphs: list[str]) -> list[list[str]]:
    preprocessed = []
    for paragraph in paragraphs:
        doc = Doc(paragraph)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)

        tokens = []
        for token in doc.tokens:
            if token.pos in ("PUNCT", "NUM"):
                continue

            token.lemmatize(morph_vocab)
            lemma = token.lemma.lower()

            if lemma not in stopwords:
                tokens.append(lemma)
        preprocessed.append(tokens)
    return preprocessed

def split_text_to_sentences(text: str) -> list[str]:
    doc = Doc(text)
    doc.segment(segmenter)
    sentences = [sent.text.strip() for sent in doc.sents if sent.text.strip()]
    return sentences

def group_sentences(sentences: list[str], group_size: int = 200) -> list[str]:
    groups = []
    for i in range(0, len(sentences), group_size):
        group = " ".join(sentences[i:i+group_size])
        groups.append(group)
    return groups

sentences = split_text_to_sentences(text)

group = group_sentences(sentences)

preprocessed_paragraphs = preprocess_text(group)

print("preprocessed_text: ", preprocessed_paragraphs[:3])

preprocessed_text:  [['преступление', 'наказание', 'федор', 'михаилович', 'достоевский', 'смерть', 'спасение', 'родион', 'раскольников', 'это', 'роман', 'петербургский', 'студент', 'родион', 'раскольников', 'бороться', 'великий', 'искушение', 'история', 'человечество', 'вернее', 'сразу', 'это', 'показаться', 'поначалу', 'странный', 'столкнуться', 'предстоять', 'каждый', 'начинать', 'задумываться', 'устройство', 'мир', 'наш', 'место', 'сущность', 'первый', 'искушение', 'стать', 'затем', 'роковой', 'замысел', 'раскольников', 'изложить', 'короткий', 'разговор', 'офицер', 'студент', 'подслушать', 'однажды', 'раскольников', 'трактир', 'речь', 'идти', 'старуха', 'процентщица', 'который', 'слово', 'студент', 'страшно', 'зло', 'скуп', 'накопить', 'огромный', 'состояние', 'весь', 'деньга', 'завещать', 'монастырь', 'вечный', 'помин', 'душа', 'студент', 'говорить', 'проклятый', 'старуха', 'убить', 'ограбить', 'уверять', 'всякий', 'зазор', 'совесть', 'объяснять', 'почему', 'сторона', 'глупый', 'бе

In [83]:
def bag_of_words(text: list[str]) -> list[int]:
    vocab_set = set()
    for tokens in text:
        vocab_set.update(tokens)
    vocabulary = sorted(list(vocab_set))

    bow_matrix = []
    for tokens in text:
        row = [0] * len(vocabulary)
        for token in tokens:
            if token in vocabulary:
                j = vocabulary.index(token)
                row[j] += 1
        bow_matrix.append(row)

    return bow_matrix


matrix = bag_of_words(preprocessed_paragraphs[:10])
print("\n--- Bag of Words ---")
print("matrix: ", matrix[:3])
for i, row in enumerate(matrix[:3]):
    print(f"{i}:", row)


--- Bag of Words ---
matrix:  [[1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 4, 2, 0, 0, 3, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 2, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 22, 1, 1, 0, 2, 4, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 0, 2, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 1, 1, 0, 0, 4, 2, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 1, 0, 8, 0, 0, 0, 1, 0, 1, 1, 1, 0, 2, 

In [84]:
def tf_idf(text: list[str]) -> tuple:
    vocab_set = set()
    for tokens in text:
        vocab_set.update(tokens)
    vocabulary = sorted(list(vocab_set))
    N = len(text)

    df = [0] * len(vocabulary)
    for i, word in enumerate(vocabulary):
        for tokens in text:
            if word in tokens:
                df[i] += 1

    idf = [math.log(N / (1 + df[i])) for i in range(len(vocabulary))]

    tfidf_matrix = []
    for tokens in text:
        total_words = len(tokens)
        word_counts = {}
        for t in tokens:
            word_counts[t] = word_counts.get(t, 0) + 1

        row = []
        for i, word in enumerate(vocabulary):
            tf = word_counts.get(word, 0) / total_words if total_words > 0 else 0
            tfidf_value = tf * idf[i]
            row.append(tfidf_value)
        tfidf_matrix.append(row)

    return vocabulary, tfidf_matrix

vocab_tfidf, tfidf_matrix = tf_idf(preprocessed_paragraphs)
print("\n--- TF-IDF ---")
print("Словарь:", vocab_tfidf[:100])
print("Матрица TF-IDF:")
for i, row in enumerate(tfidf_matrix[:3]):
    print(f"{i}:", row)



--- TF-IDF ---
Словарь: ['1789', 'confessions', 'contra', 'danke', 'i', 'ich', 'ii', 'iii', 'iv', 'pro', 'v', 'vi', 'vii', 'xix', 'а-а-а', 'абсолютно', 'авдотья', 'автор', 'адам', 'адвокатский', 'адрес', 'адрес-то', 'адский', 'ажить', 'аккуратно', 'аксиома', 'акцент', 'александр', 'алена', 'алена-то', 'али', 'аллея', 'аль', 'аля', 'амалия', 'амбиция', 'амна', 'ан', 'ана', 'анализ', 'анаполнять', 'ангел', 'англия', 'анна', 'аон', 'апожалеть', 'апосмотреть', 'аппетит', 'арестовать', 'арестовывать', 'арифметика', 'аркадий', 'армяк', 'артельный', 'аршин', 'асердце', 'асессорша', 'асестра', 'ася', 'ата', 'атака', 'атласный', 'атогда', 'атут', 'афанасиевич', 'афанасий', 'афанасий-то', 'африка', 'ах', 'ахать', 'ая', 'б', 'ба', 'баба', 'бабенка', 'бабий', 'бабушка', 'бабушкин', 'багрово-красная', 'багрово-красный', 'бакен', 'балалайка', 'балкон', 'банька', 'барк', 'барышня', 'бастилия', 'батюшка', 'бахрома', 'бахрушин', 'бахус', 'бацилла', 'башмак', 'башмаки-с', 'бег', 'бегать', 'беда', 'бедн